In [3]:
import pandas as pd
import torch

from netam import framework, models

In [4]:

shmoof_path = "/Users/matsen/data/shmoof_edges_11-Jan-2023_NoNode0_iqtree_K80+R_masked.csv"
df = pd.read_csv(shmoof_path)

df = df[df["parent"] != df["child"]]

df["sample_id"].value_counts()

sample_id
326651    22424
326713    13186
327059     4686
316188     3067
326797     3028
326780     1573
326737      442
326650      324
326907      286
Name: count, dtype: int64

In [5]:
df["nickname"] = "small"
df.loc[df["sample_id"] == 326651, "nickname"] = "51"
df.loc[df["sample_id"] == 326713, "nickname"] = "13"

df["nickname"].value_counts()

nickname
51       22424
small    13406
13       13186
Name: count, dtype: int64

In [6]:
from netam.framework import load_shmoof_dataframes

train_df, val_df = load_shmoof_dataframes(shmoof_path, val_nickname="small")

assert len(train_df) == 22424 + 13186
assert len(val_df) == 13406

In [9]:
nicknames = ["split", "51", "small", "13"]
train_losses = []
val_losses = []

kmer_length=5
max_length=410
shmoof_data_path = "/Users/matsen/data/shmoof_edges_11-Jan-2023_NoNode0_iqtree_K80+R_masked.csv"

for nickname in nicknames:
    train_df, val_df = framework.load_shmoof_dataframes(shmoof_data_path, val_nickname=nickname)
    train_dataset = framework.SHMoofDataset(train_df, kmer_length=kmer_length, max_length=max_length)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=1024, shuffle=False)
    val_dataset = framework.SHMoofDataset(val_df, kmer_length=kmer_length, max_length=max_length)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=1024, shuffle=False)

    print(f"evaluating on {nickname}...")
    model = models.SHMoofModel(train_dataset)
    burrito = framework.Burrito(train_dataset, val_dataset, model, batch_size=1024, learning_rate=0.1, min_learning_rate=1e-4, l2_regularization_coeff=1e-6)
    train_losses.append(burrito.process_data_loader(train_loader, train_mode=False))
    val_losses.append(burrito.process_data_loader(val_loader, train_mode=False))

df = pd.DataFrame({"nickname of val set": nicknames, "train_loss": train_losses, "val_loss": val_losses})
df
    

evaluating on split...
evaluating on 51...
evaluating on small...
evaluating on 13...


,nickname of val set,train_loss,val_loss
0,split,0.084404,0.085040
1,51,0.086702,0.087417
2,small,0.086756,0.073323
3,13,0.079690,0.091751
